In [1]:
# Import dependancies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import csv
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt

### Get Vaccine Data from CDC

In [2]:
# Get Vaccinations Table from CDC website
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit("https://covid.cdc.gov/covid-data-tracker/#vaccinations")
vaccine_html = browser.html
browser.click_link_by_id("vaccinations-table-toggle")
cdc_html = browser.html
cdc_parsed = bs(cdc_html, "html.parser")
table = cdc_parsed.select("table")
browser.quit()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\coled\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [3]:
# Read the CDC Vaccine HTML Table
vaccines_df = pd.read_html(str(table))[0]

In [4]:
# Change "New York State" to "New York" for data merging
vaccines_df.loc[vaccines_df["State/Territory/Federal Entity"] == "New York State", "State/Territory/Federal Entity"] = "New York"

In [5]:
# Rename "State/Territory/Federal Entity" column to "state" for data merging
vaccines_df = vaccines_df.rename(columns={"State/Territory/Federal Entity": "state"})

## Daily COVID-19 - NYT / Rearc

In [6]:
# Read Daily COVID-19 CSV from AWS S3 Bucket - Rearc / NY Times Data 
us_covid_cases_data = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-states/us-states.csv", stream=True)

In [7]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/us-covid-data.csv",  "wb") as file:
    file.write(us_covid_cases_data.content)

In [8]:
# Read US COVID Data CSV for data merging and cleaning 
us_covid_cases_df = pd.read_csv("../data/us-covid-data.csv")

# State Overview Data

In [9]:
# Get yesterday's date, which is the latest data 
today = dt.date.today()
yesterday = today - dt.timedelta(days = 1)
latest_data = yesterday.strftime("%Y-%m-%d")

In [10]:
# Get latest total cases by state for latest date
us_total_cases_to_date_df = us_covid_cases_df.groupby(["date", "state"]).sum()
us_total_cases_to_date_df.reset_index(inplace=True)
us_total_cases_to_date_df = us_total_cases_to_date_df.loc[us_total_cases_to_date_df["date"] == latest_data]

In [11]:
# Read State Lat-Lon CSV and rename columns for easier cleaning and merging
state_latlons_df = pd.read_csv("../data/statelatlong.csv")
state_latlons_df.rename(columns={"State": "abbr", "Latitude": "lat", "Longitude": "lon", "City": "state"}, inplace=True)

In [12]:
# Read State Population CSV for cleaning and merging
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")

In [13]:
# Merge, rename, and focus on defined columns
state_overview_master_df = pd.merge(state_pop_df, state_latlons_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["state", "abbr", "lat", "lon", "population"]]

In [14]:
# Merge with us_total_cases_to_date_df and focus on defined columns
state_overview_master_df = pd.merge(state_overview_master_df, us_total_cases_to_date_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths"]]

In [15]:
# Merge with Vaccination data, rename columns, focus data columns
state_overview_master_df = pd.merge(state_overview_master_df, vaccines_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df.rename(columns={"Total Doses Administered by State where Administered": "total_doses_administered", "Doses Administered per 100k by State where Administered": "doses_administered_per_100k"})
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths", "total_doses_administered", "doses_administered_per_100k"]]

In [16]:
# Add Empty Column for New Calculation
state_overview_master_df["percent_vaccinated"] = ""

In [17]:
# Calcuation for percent vaccinated by state
for index, row in state_overview_master_df.iterrows():
    population = row["population"]
    vaccinated = row["total_doses_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_overview_master_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [18]:
# Convert percent vaccinated into a float
state_overview_master_df = state_overview_master_df.astype({"percent_vaccinated": 'float64'})

In [19]:
state_overview_master_df.to_csv("../data/state-master-data.csv")

# County Overview Data

In [20]:
populations_county = pd.read_csv("../data/census-bureau-population-by-county.csv")
poverty_county = pd.read_csv("../data/poverty-and-median-household-income-data-by-us-county-2019.csv")
state_keys = pd.read_csv("../data/state-names-codes.csv")

In [21]:
poverty_county = poverty_county[["Postal Code", "county", "Poverty Estimate, All Ages", "Median Household Income"]]
poverty_county = poverty_county.rename(columns={"Postal Code": "code", "Poverty Estimate, All Ages": "poverty_est_all_ages", 
                                              "Median Household Income": "median_household_income"})
poverty_county["county"] = poverty_county["county"].str.rstrip()

In [22]:
state_keys = state_keys.rename(columns={"Code": "code", "State": "state"})
state_keys = state_keys[["state", "code"]]

In [23]:
poverty_county = pd.merge(poverty_county, state_keys, how="inner", on="code")
poverty_county.drop(columns={"state"}, inplace=True)

In [24]:
populations_county = populations_county.merge(state_keys, how="inner", on="state")
populations_county = populations_county[["code", "county", "state", "2019_population_est"]]
populations_county["code"] = populations_county["code"].str.rstrip()
populations_county["county"] = populations_county["county"].str.rstrip()

In [25]:
merged_data = populations_county.merge(poverty_county, how='inner', left_on=["code", "county"], right_on=["code", "county"])

In [26]:
merged_data.dropna(how="any", inplace=True)

### Retrieve County Cases Data

In [27]:
county_cases = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-counties/us-counties.csv", stream=True)

In [28]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/county-covid-data.csv",  "wb") as file:
    file.write(county_cases.content)

In [29]:
# Read US COVID Data CSV for data merging and cleaning 
us_county_cases = pd.read_csv("../data/county-covid-data.csv")

In [30]:
us_county_cases = us_county_cases.groupby(["date", "state", "county"]).sum()
us_county_cases.reset_index(inplace=True)

In [31]:
us_county_cases = us_county_cases.merge(merged_data, how="left")

In [32]:
us_county_cases.to_csv("../data/county-cases-daily-master.csv")

In [121]:
latest_covid_cases = us_county_cases.loc[us_county_cases["date"] == latest_data]

In [122]:
latest_covid_cases.dropna(how="any", inplace=True)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [123]:
for index, row in latest_covid_cases.iterrows():
    latest_covid_cases.at[index, "poverty_est_all_ages"] = row["poverty_est_all_ages"].replace(',', "")
    latest_covid_cases.at[index, "median_household_income"] = row["median_household_income"].replace(',', "")

In [124]:
for index, row in latest_covid_cases.iterrows():
    latest_covid_cases.at[index, "poverty_est_all_ages"] = row["poverty_est_all_ages"].replace('.', "0")
    latest_covid_cases.at[index, "median_household_income"] = row["median_household_income"].replace('.', "0")

In [125]:
latest_covid_cases.fillna(0, inplace=True)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [126]:
latest_covid_cases["poverty_est_all_ages"] = latest_covid_cases["poverty_est_all_ages"].astype('int64')
latest_covid_cases["median_household_income"] = latest_covid_cases["median_household_income"].astype('int64')

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [127]:
education_data = pd.read_csv("../data/county-level-education-stats-2015-2019.csv")

In [128]:
education_data

,state,county,less_than_high_school_diploma_2015-19,high_school_diploma_only_2015-19,some_college_or_associate_ degree_2015-19,bachelor_degree_or_higher_2015-19
0,AL,Alabama,458922.0,1022839.0,993344.0,845772.0
1,AL,Autauga,4291.0,12551.0,10596.0,9929.0
2,AL,Baldwin,13893.0,41797.0,47274.0,48148.0
3,AL,Barbour,4812.0,6396.0,4676.0,2080.0
4,AL,Bibb,3386.0,7256.0,3848.0,1678.0
...,...,...,...,...,...,...
3277,PR,Vega Baja Municipio,10494.0,9681.0,8905.0,7834.0
3278,PR,Vieques Municipio,1792.0,2440.0,875.0,1121.0
3279,PR,Villalba Municipio,3330.0,5809.0,2987.0,3015.0
3280,PR,Yabucoa Municipio,6927.0,6132.0,6494.0,4293.0


In [129]:
latest_covid_cases

,date,state,county,fips,cases,deaths,code,2019_population_est,poverty_est_all_ages,median_household_income
1099844,2021-03-06,Alabama,Autauga,1001.0,6344,92.0,AL,55869.0,6723,58233
1099845,2021-03-06,Alabama,Baldwin,1003.0,19915,289.0,AL,223234.0,22360,59871
1099846,2021-03-06,Alabama,Barbour,1005.0,2138,51.0,AL,24686.0,5909,35972
1099847,2021-03-06,Alabama,Bibb,1007.0,2460,58.0,AL,22394.0,4101,47918
1099848,2021-03-06,Alabama,Blount,1009.0,6252,128.0,AL,57826.0,9324,52902
...,...,...,...,...,...,...,...,...,...,...
1103106,2021-03-06,Wyoming,Sweetwater,56037.0,3829,36.0,WY,42343.0,3453,80639
1103107,2021-03-06,Wyoming,Teton,56039.0,3375,9.0,WY,23464.0,1396,98837
1103108,2021-03-06,Wyoming,Uinta,56041.0,2070,12.0,WY,20226.0,1699,70756
1103109,2021-03-06,Wyoming,Washakie,56043.0,886,26.0,WY,7805.0,845,55122


In [130]:
education_data.rename(columns={"state": "code"}, inplace=True)

In [131]:
education_data["county"] = education_data["county"].str.rstrip()

In [132]:
latest_covid_cases = latest_covid_cases.merge(education_data, how='outer', left_on=["code", "county"], right_on=["code", "county"])

In [133]:
latest_covid_cases.dropna(inplace=True)

In [135]:
latest_covid_cases.corr()

,fips,cases,deaths,2019_population_est,poverty_est_all_ages,median_household_income,less_than_high_school_diploma_2015-19,high_school_diploma_only_2015-19,some_college_or_associate_ degree_2015-19,bachelor_degree_or_higher_2015-19
fips,1.000000,-0.052453,-0.064357,-0.062462,-0.064708,0.044720,-0.069765,-0.063997,-0.065429,-0.065987
cases,-0.052453,1.000000,0.961780,0.814228,0.801774,0.222904,0.895886,0.727940,0.776597,0.827206
deaths,-0.064357,0.961780,1.000000,0.766462,0.764680,0.199877,0.865678,0.693500,0.723478,0.795205
2019_population_est,-0.062462,0.814228,0.766462,1.000000,0.822016,0.255144,0.823183,0.793827,0.827090,0.846171
poverty_est_all_ages,-0.064708,0.801774,0.764680,0.822016,1.000000,0.150013,0.840164,0.789074,0.801707,0.791845
median_household_income,0.044720,0.222904,0.199877,0.255144,0.150013,1.000000,0.154881,0.204959,0.236769,0.336653
less_than_high_school_diploma_2015-19,-0.069765,0.895886,0.865678,0.823183,0.840164,0.154881,1.000000,0.895227,0.906099,0.884000
high_school_diploma_only_2015-19,-0.063997,0.727940,0.693500,0.793827,0.789074,0.204959,0.895227,1.000000,0.979591,0.905287
some_college_or_associate_ degree_2015-19,-0.065429,0.776597,0.723478,0.827090,0.801707,0.236769,0.906099,0.979591,1.000000,0.939859
bachelor_degree_or_higher_2015-19,-0.065987,0.827206,0.795205,0.846171,0.791845,0.336653,0.884000,0.905287,0.939859,1.000000


In [136]:
latest_covid_cases.to_csv("../data/county-cases-latest-master.csv")

## Daily Cases

In [40]:
# Convert Dates to pd.Datetimes
us_covid_cases_df["date"] = pd.to_datetime(us_covid_cases_df["date"])

In [41]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20304,2021-03-06,Virginia,51,584537,9519
20305,2021-03-06,Washington,53,347301,5107
20306,2021-03-06,West Virginia,54,133290,2323
20307,2021-03-06,Wisconsin,55,621277,7107


In [42]:
# Create Daily New Cases Dataframe
daily_new_cases_df = us_covid_cases_df.groupby("date").sum()

In [43]:
daily_new_cases_df.reset_index(inplace=True)

In [44]:
# Add new Empty Columns for Daily New Cases and Deaths
daily_new_cases_df["daily_new_cases"] = ""
daily_new_cases_df["daily_new_deaths"] = ""

In [45]:
previous_day_cases = 0
previous_day_deaths = 0
    
for index, row in daily_new_cases_df.iterrows():
    
    # Cases
    new_cases = row["cases"] - previous_day_cases
    previous_day_cases = row["cases"]
    daily_new_cases_df.at[index, "daily_new_cases"] = new_cases
    
    # Deaths
    new_deaths = row["deaths"] - previous_day_deaths
    previous_day_deaths = row["deaths"]
    daily_new_cases_df.at[index, "daily_new_deaths"] = new_deaths

In [46]:
daily_new_cases_df["3_month_immunity_est"] = daily_new_cases_df.rolling(100)["daily_new_cases"].sum()

In [47]:
daily_new_cases_df["6_month_immunity_est"] = daily_new_cases_df.rolling(190)["daily_new_cases"].sum()

In [48]:
daily_new_cases_df = daily_new_cases_df.fillna(0)

In [49]:
daily_new_cases_df["date"] = daily_new_cases_df["date"].dt.strftime("%Y-%d-%m")

In [50]:
daily_new_cases_df

,date,fips,cases,deaths,daily_new_cases,daily_new_deaths,3_month_immunity_est,6_month_immunity_est
0,2020-21-01,53,1,0,1,0,0.0,0.0
1,2020-22-01,53,1,0,0,0,0.0,0.0
2,2020-23-01,53,1,0,0,0,0.0,0.0
3,2020-24-01,70,2,0,1,0,0.0,0.0
4,2020-25-01,76,3,0,1,0,0.0,0.0
...,...,...,...,...,...,...,...,...
406,2021-02-03,1762,28737407,515710,57789,1306,16424112.0,22983153.0
407,2021-03-03,1762,28804121,518079,66714,2369,16311093.0,23010684.0
408,2021-04-03,1762,28871536,520028,67415,1949,16200605.0,23032780.0
409,2021-05-03,1762,28937218,522511,65682,2483,16085615.0,23052852.0


In [51]:
daily_new_cases_df.to_csv("../data/state-daily-stats.csv")

### Set Up NYT Totals Master DF

In [52]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20304,2021-03-06,Virginia,51,584537,9519
20305,2021-03-06,Washington,53,347301,5107
20306,2021-03-06,West Virginia,54,133290,2323
20307,2021-03-06,Wisconsin,55,621277,7107


## Amazon RDS Updates

In [53]:
# Config Variables, and SQLalchemy
from config import endpoint, username, password
from sqlalchemy import create_engine

In [137]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [55]:
# 50 States & D.C. Data
state_overview_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)

In [56]:
engine.execute('ALTER TABLE master_table ADD PRIMARY KEY (state);')

In [57]:
# Daily Cases Data
daily_new_cases_df.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)
engine.execute('ALTER TABLE daily_new_cases ADD PRIMARY KEY (date);')

In [138]:
# County Data
latest_covid_cases.to_sql("county_cases_latest", index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE county_cases_latest ADD PRIMARY KEY (index);')

In [59]:
# Raw NTY Data
us_covid_cases_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE nyt_table ADD PRIMARY KEY (index);')